# Experimentación métodos iterativos

In [ ]:
import numpy as np
import subprocess as sp
import json

dir = "../data"

## Funciones auxiliares

In [ ]:
n = 8

def correr_algoritmo(dataset, tipo, tam, metodo, iteraciones=0, tol=0):        
    proceso = sp.run(["../src/iterativo", f"{dir}/{dataset}/matriz_{tipo}_{tam}.txt", metodo, str(iteraciones), str(tol)], capture_output=True, text=True)
    proceso.check_returncode()

    return np.array(proceso.stdout.split(" "), dtype=np.float64)

def cargar_soluciones(tipo, xs):
    for i in range(2, n + 1):
        tam = 2 ** i
        xs.append(np.genfromtxt(f"{dir}/{tipo}/vector_x_{tam}.txt"))

def cargar_soluciones_metodos(metodo):
    with open(f"{dir}/resultados_{metodo}.json", "r") as input:
        return json.load(input)
    
def guardar_resultados(data, nombre_archivo):
    with open(f"{dir}/{nombre_archivo}.json", "w") as output:
        json.dump(data, output)

## Correr experimentos

In [ ]:
max_iter = 150
step = 10
tolerancia = 1e-10

datasets = ["chico", "mediano", "grande"]
tipos = ["edd", "sim", "triang"]

## Soluciones de los métodos

### Jacobi

In [ ]:
resultados_jacobi = {}
for ds in datasets:
    resultados_jacobi[f"{ds}"] = {}
    for tipo in tipos:
        resultados_jacobi[f"{ds}"][f"{tipo}"] = []
        for i in range(2, n + 1):
            for j in range(1, max_iter + 1, step):
                res = correr_algoritmo(ds, tipo, 2 ** i, "J", j, tolerancia)
                resultados_jacobi[f"{ds}"][f"{tipo}"].append(res.tolist())

In [ ]:
guardar_resultados(resultados_jacobi, "resultados_jacobi")

### Jacobi Sumatoria

In [ ]:
resultados_jacobi_sum = {}
for ds in datasets:
    resultados_jacobi_sum[f"{ds}"] = {}
    for tipo in tipos:
        resultados_jacobi_sum[f"{ds}"][f"{tipo}"] = []
        for i in range(2, n + 1):
            for j in range(1, max_iter + 1, step):
                res = correr_algoritmo(ds, tipo, 2 ** i, "JS", j, tolerancia)
                resultados_jacobi_sum[f"{ds}"][f"{tipo}"].append(res.tolist())

In [ ]:
guardar_resultados(resultados_jacobi_sum, "resultados_jacobi_sum")

### Gauss Seidel

In [ ]:
resultados_gs = {}
for ds in datasets:
    resultados_gs[f"{ds}"] = {}
    for tipo in tipos:
        resultados_gs[f"{ds}"][f"{tipo}"] = []
        for i in range(2, n + 1):
            for j in range(1, max_iter + 1, step):
                res = correr_algoritmo(ds, tipo, 2 ** i, "GS", j, tolerancia)
                resultados_gs[f"{ds}"][f"{tipo}"].append(res.tolist())

In [ ]:
guardar_resultados(resultados_gs, "resultados_gauss_seidel")

### Gauss Seidel Sumatoria

In [ ]:
resultados_gs_sum = {}
for ds in datasets:
    resultados_gs_sum[f"{ds}"] = {}
    for tipo in tipos:
        resultados_gs_sum[f"{ds}"][f"{tipo}"] = []
        for i in range(2, n + 1):
            for j in range(1, max_iter + 1, step):
                res = correr_algoritmo(ds, tipo, 2 ** i, "GSS", j, tolerancia)
                resultados_gs_sum[f"{ds}"][f"{tipo}"].append(res.tolist())

In [ ]:
guardar_resultados(resultados_gs_sum, "resultados_gauss_seidel_sum")

### LU

In [ ]:
resultados_lu = {}
for ds in datasets:
    resultados_lu[f"{ds}"] = {}
    for tipo in tipos:
        resultados_lu[f"{ds}"][f"{tipo}"] = []
        for i in range(2, n + 1):
            res = correr_algoritmo(ds, tipo, 2 ** i, "LU")
            resultados_lu[f"{ds}"][f"{tipo}"].append(res.tolist())

In [ ]:
guardar_resultados(resultados_lu, "resultados_lu")

## Cargar vectores solución

In [ ]:
xs_chico = []
xs_mediano = []
xs_grande = []

cargar_soluciones("chico", xs_chico)
cargar_soluciones("mediano", xs_mediano)
cargar_soluciones("grande", xs_grande)

## Cargar soluciones métodos

In [ ]:
sol_j = cargar_soluciones_metodos("jacobi")
sol_js = cargar_soluciones_metodos("jacobi_sum")
sol_gs = cargar_soluciones_metodos("gauss_seidel")
sol_gss = cargar_soluciones_metodos("gauss_seidel_sum")
sol_lu = cargar_soluciones_metodos("lu")

## Generación de Gráficos

### Error

In [ ]:
errores = {}
offset = (max_iter + 1) // step 
for tipo in sol_j["chico"].values():
    j = 0
    for solucion in xs_chico:
        n = len(solucion)
        errores[f"{n}"] = []
        for i in range((max_iter + 1) // step):
            errores[f"{n}"].append(np.linalg.norm(solucion - np.array(tipo[i + j * offset], dtype=np.float64)))
        j += 1

errores

### Tiempo de cómputo

### LU

### LU vs Métodos Iterativos

- Error de aproximacion entre valor actual y final (por cantidad de iteraciones) para matrices de diferentes tamaños
- Tiempo final de computo por iteraciones y por tamaño de matrices
- LU tiempo de computo por tamaño de matriz. Error numerico (comparar con x real).
- Comparar LU con todos los metodos iterativos.